In [1]:
%pip install torch
%pip install numpy
%pip install pandas
%pip install scikit-learn
%pip install coremltools

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [40]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from numpy import genfromtxt
from sklearn.preprocessing import LabelEncoder

In [41]:
# Set up the paths
HOME_PATH = os.path.expanduser('~')
MODELS_PATH = f'{HOME_PATH}/Developer/BU/research/models'
DATASET_PATH = f'../../../data/'
data_features = f'{DATASET_PATH}/WISDM_x.csv'
data_labels = f'{DATASET_PATH}/WISDM_y.csv'

In [3]:
# Load Data
x = genfromtxt(data_features, delimiter=',')
y_df = pd.read_csv(data_labels)
y = y_df.values.flatten()  # Flatten if y is 2D

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Function to create time series dataset
def create_series(x, y, timestep, overlap):
    slide_step = int(timestep * (1 - overlap))
    data_num = int((len(x) / slide_step) - 1)
    dataset = np.ndarray(shape=(data_num, timestep, x.shape[1]))
    labels = []

    for i in range(data_num):
        labels.append(y[slide_step * (i + 1) - 1])
        for j in range(timestep):
            dataset[i, j, :] = x[slide_step * i + j, :]

    return dataset, np.array(labels)

# Create time series
timestep = 16  # Replace with your value
overlap = 0.5  # Replace with your value
X_series, y_series = create_series(x, y_encoded, timestep, overlap)

In [4]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_series, y_series, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
x_train_tensor = torch.tensor(X_train, dtype=torch.float32)
x_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)



In [5]:
import torch
import torch.nn as nn

class GRUNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(GRUNet, self).__init__()
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2

        self.gru1 = nn.GRU(input_size, hidden_size1, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.gru2 = nn.GRU(hidden_size1, hidden_size2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(1, x.size(0), self.hidden_size1).to(x.device)
        
        # First GRU layer
        out, _ = self.gru1(x, h0)
        out = self.dropout1(out)
        
        # Second GRU layer
        h1 = torch.zeros(1, x.size(0), self.hidden_size2).to(x.device)
        out, _ = self.gru2(out, h1)
        out = self.dropout2(out)

        # Dense layer
        out = self.fc(out[:, -1, :])  # Taking the last time step
        return out


In [6]:
# Parameters
input_size = 3  # Number of features
hidden_size1 = 50
hidden_size2 = 25
output_size = 6
n_steps = 16

# Create the model
model = GRUNet(input_size, hidden_size1, hidden_size2, output_size)

# Training setup (for demonstration)
# Define your dataset here
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.int64))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [7]:
def train(model, train_loader, loss_fn, optimizer, epochs=0):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(y_pred.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        avg_loss = running_loss / len(train_loader)
        accuracy = 100 * correct / total
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

train(model, train_loader, loss_fn, optimizer, epochs=20)


Epoch [1/20], Loss: 0.5832, Accuracy: 79.70%
Epoch [2/20], Loss: 0.3713, Accuracy: 86.83%
Epoch [3/20], Loss: 0.3040, Accuracy: 89.52%
Epoch [4/20], Loss: 0.2685, Accuracy: 90.95%
Epoch [5/20], Loss: 0.2435, Accuracy: 91.88%
Epoch [6/20], Loss: 0.2249, Accuracy: 92.47%
Epoch [7/20], Loss: 0.2139, Accuracy: 92.90%
Epoch [8/20], Loss: 0.2015, Accuracy: 93.33%
Epoch [9/20], Loss: 0.1940, Accuracy: 93.62%
Epoch [10/20], Loss: 0.1854, Accuracy: 93.87%
Epoch [11/20], Loss: 0.1801, Accuracy: 94.13%
Epoch [12/20], Loss: 0.1760, Accuracy: 94.26%
Epoch [13/20], Loss: 0.1688, Accuracy: 94.49%
Epoch [14/20], Loss: 0.1638, Accuracy: 94.63%
Epoch [15/20], Loss: 0.1595, Accuracy: 94.84%
Epoch [16/20], Loss: 0.1575, Accuracy: 94.89%
Epoch [17/20], Loss: 0.1510, Accuracy: 95.09%
Epoch [18/20], Loss: 0.1487, Accuracy: 95.15%
Epoch [19/20], Loss: 0.1459, Accuracy: 95.23%
Epoch [20/20], Loss: 0.1431, Accuracy: 95.40%


In [28]:
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.int64))
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
def evaluate(model, test_loader, loss_fn):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            total_loss += loss_fn(y_pred, y_batch).item()
            _, predicted = torch.max(y_pred.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total
    return avg_loss, accuracy

avg_loss, accuracy = evaluate(model, test_loader, loss_fn)
print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")



Test Loss: 0.1640, Test Accuracy: 0.9473


In [42]:
model_path = f'{MODELS_PATH}/gru_base.pth'
torch.save(model.state_dict(), model_path)

### Coreml metrics

In [43]:
import coremltools as ct
example_input = torch.rand(1,16, 3) 

model.eval()
traced_model = torch.jit.trace(model, example_input)
out = traced_model(example_input)

# Convert to Core ML program using the Unified Conversion API.
coreml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[ct.TensorType(shape=example_input.shape)]
 )

# Save the converted model.
coreml_model.save(f'{MODELS_PATH}/gru.mlpackage')

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 2894.95 passes/s]


In [44]:
coreml_model

input {
  name: "x"
  type {
    multiArrayType {
      shape: 1
      shape: 16
      shape: 3
      dataType: FLOAT32
    }
  }
}
output {
  name: "linear_24"
  type {
    multiArrayType {
      shape: 1
      shape: 6
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}

In [45]:
from pathlib import Path

def coreml_metrics(model_name, X_test, y_test, model_path):
    predictions = []
    for id in range(len(X_test)):
        X_test_new = np.expand_dims(X_test[id], axis=0)
        output_dict = model_name.predict({'x': X_test_new})
        pred_class = np.argmax(output_dict['linear_24'])
       
        predictions.append(pred_class)
    
    accuracy = np.sum(predictions == y_test) / len(predictions)
    print("Accuracy:", accuracy)
    
    model_file = Path(model_path)
    
    # Size in bytes
    model_size_bytes = model_file.stat().st_size
    
    # Convert size to kilobytes (optional)
    model_size_kb = model_size_bytes / 1024
    print(f"Size of the model: {model_size_kb:.2f} KB")

In [46]:
import coremltools as ct
import coremltools.optimize.coreml as cto

coreml_model = ct.models.MLModel(f'{MODELS_PATH}/gru.mlpackage')
coreml_model

input {
  name: "x"
  type {
    multiArrayType {
      shape: 1
      shape: 16
      shape: 3
      dataType: FLOAT32
    }
  }
}
output {
  name: "linear_24"
  type {
    multiArrayType {
      shape: 1
      shape: 6
      dataType: FLOAT32
    }
  }
}
metadata {
  userDefined {
    key: "com.github.apple.coremltools.source"
    value: "torch==2.0.0"
  }
  userDefined {
    key: "com.github.apple.coremltools.source_dialect"
    value: "TorchScript"
  }
  userDefined {
    key: "com.github.apple.coremltools.version"
    value: "7.1"
  }
}

In [47]:
model_name = coreml_model
model_path = f'{MODELS_PATH}/gru.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9472800793469138
Size of the model: 0.12 KB


## Post Training Optimization

In [51]:
from coremltools.optimize.coreml import (
    OpThresholdPrunerConfig,
    OpMagnitudePrunerConfig,
    OpPalettizerConfig,
    OpLinearQuantizerConfig,
    OptimizationConfig,
    prune_weights,
)

### Quantization

In [52]:
op_config = OpLinearQuantizerConfig(
    mode="linear_symmetric", weight_threshold=512
)
config = OptimizationConfig(global_config=op_config)

compressed_8_bit_model = cto.linear_quantize_weights(coreml_model, config=config)

Running compression pass linear_quantize_weights: 0 ops [00:00, ? ops/s]
Running compression pass linear_quantize_weights: 100%|██████████| 12/12 [00:00<00:00, 3425.79 ops/s]
Running compression pass linear_quantize_weights: 0 ops [00:00, ? ops/s]
Running compression pass linear_quantize_weights: 100%|██████████| 12/12 [00:00<00:00, 2649.59 ops/s]
Running compression pass linear_quantize_weights:   0%|          | 0/3 [00:00<?, ? ops/s]/opt/homebrew/lib/python3.10/site-packages/coremltools/optimize/coreml/_quantization_passes.py:686: RuntimeWarning: invalid value encountered in divide
  quantized_data = np.round(original_data / scale)
/opt/homebrew/lib/python3.10/site-packages/coremltools/optimize/coreml/_quantization_passes.py:690: RuntimeWarning: invalid value encountered in cast
  quantized_data = np.clip(quantized_data, q_val_min, q_val_max).astype(np_dtype)
Running compression pass linear_quantize_weights: 100%|██████████| 3/3 [00:00<00:00, 3811.85 ops/s]
Running MIL frontend_milin

In [53]:
compressed_8_bit_model.save(f'{MODELS_PATH}/gru_8bitQuantized_mlmodel.mlpackage')

In [54]:
model_name = compressed_8_bit_model
model_path = f'{MODELS_PATH}/gru_8bitQuantized_mlmodel.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9472419317921721
Size of the model: 0.12 KB


### Pruning

#### a) OpMagnitudePrunerConfig: Prune the weights with a constant sparsity percentile

In [55]:
op_config = OpMagnitudePrunerConfig(
    target_sparsity=0.6,
    weight_threshold=1024,
)
config = OptimizationConfig(global_config=op_config)
gru_magnitude_pruner = prune_weights(coreml_model, config=config)


Running compression pass prune_weights: 0 ops [00:00, ? ops/s]
Running compression pass prune_weights: 100%|██████████| 12/12 [00:00<00:00, 3222.26 ops/s]
Running compression pass prune_weights: 0 ops [00:00, ? ops/s]
Running compression pass prune_weights: 100%|██████████| 3/3 [00:00<00:00, 53773.13 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1822.55 passes/s]


In [56]:
gru_magnitude_pruner.save(f'{MODELS_PATH}/gru_magnitude_pruner.mlpackage')

In [57]:
model_name = gru_magnitude_pruner
model_path = f'{MODELS_PATH}/gru_magnitude_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.8627450980392157
Size of the model: 0.12 KB


#### b) OpThresholdPrunerConfig: Sets all weight values below a certain value.

In [58]:
op_config = OpThresholdPrunerConfig(
    threshold=0.001,
    minimum_sparsity_percentile=0.01,
    weight_threshold=1024,
)

config = OptimizationConfig(global_config=op_config)
gru_threshold_pruner = prune_weights(coreml_model, config=config)

Running compression pass prune_weights: 0 ops [00:00, ? ops/s]
Running compression pass prune_weights:   0%|          | 0/12 [00:00<?, ? ops/s]weight value has sparsity of 0.0028 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0016 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0028 < minimum_sparsity_percentile 0.01. Skipped.
Running compression pass prune_weights: 100%|██████████| 12/12 [00:00<00:00, 9057.34 ops/s]
Running compression pass prune_weights: 0 ops [00:00, ? ops/s]
Running compression pass prune_weights:   0%|          | 0/12 [00:00<?, ? ops/s]weight value has sparsity of 0.0048 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0008 < minimum_sparsity_percentile 0.01. Skipped.
weight value has sparsity of 0.0048 < minimum_sparsity_percentile 0.01. Skipped.
Running compression pass prune_weights: 100%|██████████| 3/3 [00:00<00:00, 48395.82 ops/s]
Running MIL frontend_milinternal pipeline: 0 

In [59]:
gru_threshold_pruner.save(f'{MODELS_PATH}/gru_threshold_pruner.mlpackage')

In [60]:
model_name = gru_threshold_pruner
model_path = f'{MODELS_PATH}/gru_threshold_pruner.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.9472800793469138
Size of the model: 0.12 KB


### Palletization

In [61]:
op_config = OpPalettizerConfig(
    mode="kmeans", 
    nbits=6
)

config = OptimizationConfig(global_config=op_config)
gru_palettizer = cto.palettize_weights(coreml_model, config=config)

Running compression pass palettize_weights: 0 ops [00:00, ? ops/s]
Running compression pass palettize_weights: 100%|██████████| 12/12 [00:00<00:00, 36.64 ops/s]
Running compression pass palettize_weights: 0 ops [00:00, ? ops/s]
Running compression pass palettize_weights: 100%|██████████| 3/3 [00:00<00:00, 14546.72 ops/s]
Running MIL frontend_milinternal pipeline: 0 passes [00:00, ? passes/s]
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 1802.32 passes/s]


In [62]:
gru_palettizer.save(f'{MODELS_PATH}/gru_palettizer.mlpackage')

In [63]:
model_name = gru_palettizer
model_path = f'{MODELS_PATH}/gru_palettizer.mlpackage'

coreml_metrics(model_name, X_test, y_test, model_path)

Accuracy: 0.946669718471046
Size of the model: 0.12 KB
